In [32]:
import transformers

print(transformers.__version__)

4.42.0.dev0


In [33]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
batch_size = 16

In [34]:
from datasets import load_dataset, load_metric

In [35]:
datasets = load_dataset("ai4privacy/pii-masking-300k")
print("-------------------------")
print(datasets)
print("----------------------")
print(datasets["train"][0]["source_text"])
print("---------------------------")
print(datasets["train"][0]["mbert_bio_labels"])
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_input = tokenizer(datasets["train"][0]["source_text"])
print("-------------------------")
print(tokenized_input)
print("-------------------------")
print(f"{len(datasets['train'][0]['mbert_bio_labels'])} vs {len(tokenized_input['input_ids'])}")
print("-------------------------")
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print("-------------------------")
print(f"{len(tokens)}")

-------------------------
DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})
----------------------
Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:

- wynqvrh053 - Meeting at 10:20am
- luka.burg - Meeting at 21
- qahil.wittauer - Meeting at quarter past 13
- gholamhossein.ruschke - Meeting at 9:47 PM
- pdmjrsyoz1460 
---------------------------
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [36]:
label_list = [
    'O',
    'B-BOD',
    'I-BOD',
    'B-BUILDING',
    'I-BUILDING',
    'B-CARDISSUER',
    'I-CARDISSUER',
    'B-CITY',
    'I-CITY',
    'B-COUNTRY',
    'I-COUNTRY',
    'B-DATE',
    'I-DATE',
    'B-DRIVERLICENSE',
    'I-DRIVERLICENSE',
    'B-EMAIL',
    'I-EMAIL',
    'B-GEOCOORD',
    'I-GEOCOORD',
    'B-GIVENNAME1',
    'I-GIVENNAME1',
    'B-GIVENNAME2',
    'I-GIVENNAME2',
    'B-IDCARD',
    'I-IDCARD',
    'B-IP',
    'I-IP',
    'B-LASTNAME1',
    'I-LASTNAME1',
    'B-LASTNAME2',
    'I-LASTNAME2',
    'B-LASTNAME3',
    'I-LASTNAME3',
    'B-PASS',
    'I-PASS',
    'B-PASSPORT',
    'I-PASSPORT',
    'B-POSTCODE',
    'I-POSTCODE',
    'B-SECADDRESS',
    'I-SECADDRESS',
    'B-SEX',
    'I-SEX',
    'B-SOCIALNUMBER',
    'I-SOCIALNUMBER',
    'B-STATE',
    'I-STATE',
    'B-STREET',
    'I-STREET',
    'B-TEL',
    'I-TEL',
    'B-TIME',
    'I-TIME',
    'B-TITLE',
    'I-TITLE',
    'B-USERNAME',
    'I-USERNAME',
]

id2label = {i:label for i, label in enumerate(label_list)}
label2id = {label:i for i, label in enumerate(label_list)}

In [37]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [39]:
show_random_elements(datasets["train"])

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set
0,"""1993-06-09T00:00:00, jupettmann, jupettmann@protonmail.com, 756 7650 2377 83, 5888509152, S1389882, Schweiz, 780, Weiherstrasse, Fehren, Solothurn, 4232, 348b:a00d:9b6e:4698:31aa:b42f:b14b:f6d8, United Kingdom, 633, Montague Road, Portsmouth, ENG, PO2, Schweiz, 288, Fauchy, Mont-la-Ville, VD, 1148, CH, 436, Vorstadt, Hägglingen, Aargau, 5607, Villa 244, Schweiz, 372, Sant'Antonio, Lavizzara, Tessin, 66","""[BOD], [USERNAME], [EMAIL], [SOCIALNUMBER], [DRIVERLICENSE], [PASSPORT], [COUNTRY], [BUILDING], [STREET], [CITY], [STATE], [POSTCODE], [IP], [COUNTRY], [BUILDING], [STREET], [CITY], [STATE], [POSTCODE], [COUNTRY], [BUILDING], [STREET], [CITY], [STATE], [POSTCODE], [COUNTRY], [BUILDING], [STREET], [CITY], [STATE], [POSTCODE], [SECADDRESS], [COUNTRY], [BUILDING], [STREET], [CITY], [STATE], 66","[{'value': '1993-06-09T00:00:00', 'start': 1, 'end': 20, 'label': 'BOD'}, {'value': 'jupettmann', 'start': 22, 'end': 32, 'label': 'USERNAME'}, {'value': 'jupettmann@protonmail.com', 'start': 34, 'end': 59, 'label': 'EMAIL'}, {'value': '756 7650 2377 83', 'start': 61, 'end': 77, 'label': 'SOCIALNUMBER'}, {'value': '5888509152', 'start': 79, 'end': 89, 'label': 'DRIVERLICENSE'}, {'value': 'S1389882', 'start': 91, 'end': 99, 'label': 'PASSPORT'}, {'value': 'Schweiz', 'start': 101, 'end': 108, 'label': 'COUNTRY'}, {'value': '780', 'start': 110, 'end': 113, 'label': 'BUILDING'}, {'value': 'Weiherstrasse', 'start': 115, 'end': 128, 'label': 'STREET'}, {'value': 'Fehren', 'start': 130, 'end': 136, 'label': 'CITY'}, {'value': 'Solothurn', 'start': 138, 'end': 147, 'label': 'STATE'}, {'value': '4232', 'start': 149, 'end': 153, 'label': 'POSTCODE'}, {'value': '348b:a00d:9b6e:4698:31aa:b42f:b14b:f6d8', 'start': 155, 'end': 194, 'label': 'IP'}, {'value': 'United Kingdom', 'start': 196, 'end': 210, 'label': 'COUNTRY'}, {'value': '633', 'start': 212, 'end': 215, 'label': 'BUILDING'}, {'value': 'Montague Road', 'start': 217, 'end': 230, 'label': 'STREET'}, {'value': 'Portsmouth', 'start': 232, 'end': 242, 'label': 'CITY'}, {'value': 'ENG', 'start': 244, 'end': 247, 'label': 'STATE'}, {'value': 'PO2', 'start': 249, 'end': 252, 'label': 'POSTCODE'}, {'value': 'Schweiz', 'start': 254, 'end': 261, 'label': 'COUNTRY'}, {'value': '288', 'start': 263, 'end': 266, 'label': 'BUILDING'}, {'value': 'Fauchy', 'start': 268, 'end': 274, 'label': 'STREET'}, {'value': 'Mont-la-Ville', 'start': 276, 'end': 289, 'label': 'CITY'}, {'value': 'VD', 'start': 291, 'end': 293, 'label': 'STATE'}, {'value': '1148', 'start': 295, 'end': 299, 'label': 'POSTCODE'}, {'value': 'CH', 'start': 301, 'end': 303, 'label': 'COUNTRY'}, {'value': '436', 'start': 305, 'end': 308, 'label': 'BUILDING'}, {'value': 'Vorstadt', 'start': 310, 'end': 318, 'label': 'STREET'}, {'value': 'Hägglingen', 'start': 320, 'end': 330, 'label': 'CITY'}, {'value': 'Aargau', 'start': 332, 'end': 338, 'label': 'STATE'}, {'value': '5607', 'start': 340, 'end': 344, 'label': 'POSTCODE'}, {'value': 'Villa 244', 'start': 346, 'end': 355, 'label': 'SECADDRESS'}, {'value': 'Schweiz', 'start': 357, 'end': 364, 'label': 'COUNTRY'}, {'value': '372', 'start': 366, 'end': 369, 'label': 'BUILDING'}, {'value': 'Sant'Antonio', 'start': 371, 'end': 383, 'label': 'STREET'}, {'value': 'Lavizzara', 'start': 385, 'end': 394, 'label': 'CITY'}, {'value': 'Tessin', 'start': 396, 'end': 402, 'label': 'STATE'}]","[[396, 402, ""STATE""], [385, 394, ""CITY""], [371, 383, ""STREET""], [366, 369, ""BUILDING""], [357, 364, ""COUNTRY""], [346, 355, ""SECADDRESS""], [340, 344, ""POSTCODE""], [332, 338, ""STATE""], [320, 330, ""CITY""], [310, 318, ""STREET""], [305, 308, ""BUILDING""], [301, 303, ""COUNTRY""], [295, 299, ""POSTCODE""], [291, 293, ""STATE""], [276, 289, ""CITY""], [268, 274, ""STREET""], [263, 266, ""BUILDING""], [254, 261, ""COUNTRY""], [249, 252, ""POSTCODE""], [244, 247, ""STATE""], [232, 242, ""CITY""], [217, 230, ""STR

In [40]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [41]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [42]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 31178, 117, 10531, 10124, 10464, 49219, 106, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [43]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 31178, 117, 10531, 10124, 10464, 49219, 24137, 10708, 21296, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [44]:
label_all_tokens = True

In [58]:
def filter_inputs(example, idx):
    if idx >= 500:
        return False
    
    if example["language"] != "English":
        return False
    
    tokenized = tokenizer(example["source_text"], padding=True, truncation=True)
    tokenized_words = tokenizer.convert_ids_to_tokens(tokenized["input_ids"])
    tokenized_words = tokenized_words[1:-1] # remove CLS and SEP

    # global largest_token_size
    # tokenized_len = len(tokenized_words)
    # if tokenized_len > largest_token_size:
    #     largest_token_size = tokenized_len
    if tokenized_words != example["mbert_text_tokens"]:
        print(f"EXCLUDING ID={example['id']} because tokenized version does not match example mbert tokens.")
        return False

    return True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["source_text"], padding=True, truncation=True)

    # labels = []
    # for batch_index, label in enumerate(examples["mbert_text_tokens"]):
    #     word_ids = tokenized_inputs.word_ids(batch_index=batch_index)

        # label_ids = []
    # ctr = 0

    all_labels = []
    
    for idx, labels in enumerate(examples['mbert_bio_labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        label_ids = []
        ctr = 0
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                expected_mbert_label = labels[ctr]
                label_ids.append(label2id[expected_mbert_label])
                ctr += 1
                
        all_labels.append(label_ids)
        # example["labels"] = labels
    # # Map each input_id (101, 23533, 2121, etc) into the expected mbert_bio_label ID (using label2id or what have you)
    # for input_id in tokenized_inputs["input_ids"]:
    #     if input_id == 101 or input_id == 102:
    #         # 101 = CLS, 102 = SEP, label id should be -100
    #         label_ids.append(-100)
    #     else:
    #         expected_mbert_label = examples["mbert_bio_labels"][ctr]
    #         label_ids.append(label2id[expected_mbert_label])
    #         # print(label2id[expected_mbert_label])
    #         # print(expected_mbert_label)
    #         ctr += 1

    tokenized_inputs["labels"] = all_labels
    
    return tokenized_inputs

    # tokenized_words = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
    # tokenized_words = tokenized_words[1:-1] # remove CLS and SEP

    # if tokenized_words != examples["mbert_text_tokens"]:
    #     # raise ValueError(f"Example ID={examples['id']} has mismatched tokenizer words compared to example words.\nlen(Tokenized)={len(tokenized_words)}, len(Examples)={len(examples['mbert_text_tokens'])}\n\nTokenized={tokenized_words}\n\nExample={examples['mbert_text_tokens']}")
    #     return None



# def tokenize_and_align_labels(examples):
#     tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

#     labels = []
#     for i, label in enumerate(examples[f"{task}_tags"]):
#         word_ids = tokenized_inputs.word_ids(batch_index=i)
#         previous_word_idx = None
#         label_ids = []
#         for word_idx in word_ids:
#             # Special tokens have a word id that is None. We set the label to -100 so they are automatically
#             # ignored in the loss function.
#             if word_idx is None:
#                 label_ids.append(-100)
#             # We set the label for the first token of each word.
#             elif word_idx != previous_word_idx:
#                 label_ids.append(label[word_idx])
#             # For the other tokens in a word, we set the label to either the current label or -100, depending on
#             # the label_all_tokens flag.
#             else:
#                 label_ids.append(label[word_idx] if label_all_tokens else -100)
#             previous_word_idx = word_idx

#         labels.append(label_ids)

#     tokenized_inputs["labels"] = labels
#     return tokenized_inputs

In [59]:
#datasets.filter(filter_inputs, with_indices=True, batched=True)
# datasets.filter(filter_inputs, batched=True)
tokenized_datasets = datasets.filter(filter_inputs, with_indices=True).map(tokenize_and_align_labels, batched=True)
# tokenize_and_align_labels(datasets['train'][:10])

Filter:   0%|          | 0/177677 [00:00<?, ? examples/s]

Filter:   0%|          | 0/47728 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [60]:
# tokenized_datasets["train"][0]


In [61]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
print(label_list)
model

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['O', 'B-BOD', 'I-BOD', 'B-BUILDING', 'I-BUILDING', 'B-CARDISSUER', 'I-CARDISSUER', 'B-CITY', 'I-CITY', 'B-COUNTRY', 'I-COUNTRY', 'B-DATE', 'I-DATE', 'B-DRIVERLICENSE', 'I-DRIVERLICENSE', 'B-EMAIL', 'I-EMAIL', 'B-GEOCOORD', 'I-GEOCOORD', 'B-GIVENNAME1', 'I-GIVENNAME1', 'B-GIVENNAME2', 'I-GIVENNAME2', 'B-IDCARD', 'I-IDCARD', 'B-IP', 'I-IP', 'B-LASTNAME1', 'I-LASTNAME1', 'B-LASTNAME2', 'I-LASTNAME2', 'B-LASTNAME3', 'I-LASTNAME3', 'B-PASS', 'I-PASS', 'B-PASSPORT', 'I-PASSPORT', 'B-POSTCODE', 'I-POSTCODE', 'B-SECADDRESS', 'I-SECADDRESS', 'B-SEX', 'I-SEX', 'B-SOCIALNUMBER', 'I-SOCIALNUMBER', 'B-STATE', 'I-STATE', 'B-STREET', 'I-STREET', 'B-TEL', 'I-TEL', 'B-TIME', 'I-TIME', 'B-TITLE', 'I-TITLE', 'B-USERNAME', 'I-USERNAME']


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

In [62]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    # num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
)

In [63]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [64]:
import evaluate
metric = evaluate.load("seqeval")

In [65]:
# labels = [label_list[i] for i in example[f"{task}_tags"]]
# metric.compute(predictions=[labels], references=[labels])

In [66]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    # print(f"predictions.shape: {predictions.shape}")
    # print(f"labels.shape: {labels.shape}")
    
    predictions = np.argmax(predictions, axis=2)
    
    # print(f"predictions.shape after argmax: {predictions.shape}")
    # print(f"{predictions}")
    
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # print(f"true_predictions:\n{true_predictions}")
    # print(f"true_labels:\n{true_labels}")
    # for idx, itm in enumerate(true_labels):
    #     labels_set = set(true_labels[idx])
    #     preds_set = set(true_predictions[idx])
    #     print(f"diff @ {idx}:\n{labels_set - preds_set}")
 

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [67]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [68]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,3.725668,0.000096,0.001468,0.000180,0.222301
2,No log,3.451936,0.000000,0.000000,0.000000,0.635950
3,No log,3.244551,0.000000,0.000000,0.000000,0.729189
4,No log,3.112367,0.000000,0.000000,0.000000,0.749023
5,No log,3.065049,0.000000,0.000000,0.000000,0.752932


TrainOutput(global_step=80, training_loss=3.421538543701172, metrics={'train_runtime': 30.9025, 'train_samples_per_second': 80.899, 'train_steps_per_second': 2.589, 'total_flos': 182636458290000.0, 'train_loss': 3.421538543701172, 'epoch': 5.0})

In [69]:
trainer.evaluate()

{'eval_loss': 3.065049409866333,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.7529317526433835,
 'eval_runtime': 2.5977,
 'eval_samples_per_second': 192.477,
 'eval_steps_per_second': 6.159,
 'epoch': 5.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [70]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'BOD': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 154},
 'BUILDING': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 88},
 'CARDISSUER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'CITY': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 90},
 'COUNTRY': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 66},
 'DATE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 98},
 'DRIVERLICENSE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 116},
 'EMAIL': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 142},
 'GEOCOORD': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 19},
 'GIVENNAME1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 129},
 'GIVENNAME2': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 34},
 'IDCARD': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 116},
 'IP': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 115},
 'LASTNAME1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '

In [71]:
trainer.save_model()

In [72]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch

text = "My name is John Smith."

classifier = pipeline("ner", model=f"{model_name}-finetuned-{task}")
classifier(text)

[{'entity': 'LABEL_0',
  'score': 0.0328758,
  'index': 1,
  'word': 'My',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_50',
  'score': 0.029664248,
  'index': 2,
  'word': 'name',
  'start': 3,
  'end': 7},
 {'entity': 'LABEL_0',
  'score': 0.029966153,
  'index': 3,
  'word': 'is',
  'start': 8,
  'end': 10},
 {'entity': 'LABEL_0',
  'score': 0.034548488,
  'index': 4,
  'word': 'John',
  'start': 11,
  'end': 15},
 {'entity': 'LABEL_0',
  'score': 0.034664508,
  'index': 5,
  'word': 'Smith',
  'start': 16,
  'end': 21},
 {'entity': 'LABEL_0',
  'score': 0.028726818,
  'index': 6,
  'word': '.',
  'start': 21,
  'end': 22}]